SetUp

In [1]:
!pip install openai requests tiktoken numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Retrieve key and endpoint

In [2]:
import os
import openai
from dotenv import load_dotenv
#load variable from .env file
load_dotenv()
# Set the environment variable
load_dotenv()
azure_oai_endpoint = os.getenv("AZURE_OAI_FINETUNE_ENDPOINT")
azure_oai_key = os.getenv("AZURE_OAI_FINETUNE_KEY")
azure_oai_deployment = os.getenv("AZURE_OAI_FINETUNE_DEPLOYMENT")


# Verify that it has been set
print(azure_oai_endpoint)
# Verify that it has been set
print(azure_oai_key)


https://finetuneai.openai.azure.com/
52764c4f08f446f392342880499e7434


Import Data

In [5]:
# Run preliminary checks

import json

# Load the training set
with open(r'C:\Users\21629\Desktop\new\train_data.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open(r'C:\Users\21629\Desktop\new\val_data.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 45
First example in training set:
{'role': 'system', 'content': 'You are a helper in fianance and an advisor in investement'}
{'role': 'user', 'content': 'How do I build credit?'}
{'role': 'assistant', 'content': 'Borrow money and actually pay it back. Revolutionary, isn’t it?'}

Number of examples in validation set: 12
First example in validation set:
{'role': 'system', 'content': 'You are a helper in fianance and an advisor in investement'}
{'role': 'user', 'content': 'How do I save for retirement?'}
{'role': 'assistant', 'content': "Just start putting money away now and hope you don't need it until you're 80."}


In [6]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        if not isinstance(message, dict):
            print(f"Unexpected message format: {message}")
            continue
        num_tokens += tokens_per_message
        for key, value in message.items():
            if not isinstance(value, str):
                print(f"Unexpected value type for key '{key}': {value}")
                continue
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if not isinstance(message, dict):
            print(f"Unexpected message format: {message}")
            continue
        if message.get("role") == "assistant":
            content = message.get("content", "")
            if not isinstance(content, str):
                print(f"Unexpected content type: {content}")
                continue
            num_tokens += len(encoding.encode(content))
    return num_tokens

def print_distribution(values, name):
    if values:
        print(f"\n#### Distribution of {name}:")
        print(f"min / max: {min(values)}, {max(values)}")
        print(f"mean / median: {np.mean(values)}, {np.median(values)}")
        print(f"p5 / p95: {np.quantile(values, 0.05)}, {np.quantile(values, 0.95)}")
    else:
        print(f"No values to display for {name}")

files = [
    r'C:\Users\21629\Desktop\new\train_data.jsonl',
    r'C:\Users\21629\Desktop\new\val_data.jsonl'
]

for file in files:
    print(f"Processing file: {file}")
    try:
        with open(file, 'r', encoding='utf-8') as f:
            total_tokens = []
            assistant_tokens = []
            for line in f:
                try:
                    ex = json.loads(line)
                    messages = ex.get("messages", [])
                    if not isinstance(messages, list):
                        raise ValueError("The 'messages' field should be a list.")
                    total_tokens.append(num_tokens_from_messages(messages))
                    assistant_tokens.append(num_assistant_tokens_from_messages(messages))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON line: {line}")
                except ValueError as ve:
                    print(f"ValueError: {ve} - line: {line}")
                except Exception as e:
                    print(f"Unexpected error processing line: {e} - line: {line}")

            if total_tokens and assistant_tokens:
                print_distribution(total_tokens, "total tokens")
                print_distribution(assistant_tokens, "assistant tokens")
            else:
                print("No valid data to process.")

            print('*' * 50)

    except FileNotFoundError:
        print(f"File not found: {file}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Processing file: C:\Users\21629\Desktop\new\train_data.jsonl

#### Distribution of total tokens:
min / max: 46, 61
mean / median: 52.044444444444444, 52.0
p5 / p95: 46.0, 60.599999999999994

#### Distribution of assistant tokens:
min / max: 9, 26
mean / median: 16.88888888888889, 17.0
p5 / p95: 12.0, 24.799999999999997
**************************************************
Processing file: C:\Users\21629\Desktop\new\val_data.jsonl

#### Distribution of total tokens:
min / max: 46, 58
mean / median: 52.333333333333336, 52.5
p5 / p95: 47.1, 57.45

#### Distribution of assistant tokens:
min / max: 14, 25
mean / median: 18.416666666666668, 18.0
p5 / p95: 14.0, 24.45
**************************************************


In [7]:
import os
from openai import AzureOpenAI

# Initialize AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint=azure_oai_endpoint,
    api_key=azure_oai_key,
    api_version="2024-05-01-preview"  # Ensure this API version is correct
)

training_file_name = r'C:\Users\21629\Desktop\new\train_data.jsonl'
validation_file_name = r'C:\Users\21629\Desktop\new\val_data.jsonl'

try:
    # Upload the training dataset file
    with open(training_file_name, "rb") as file:
        training_response = client.files.create(
            file=file, purpose="fine-tune"
        )
    training_file_id = training_response.id
    print("Training file ID:", training_file_id)

except Exception as e:
    print(f"Error uploading training file: {e}")

try:
    # Upload the validation dataset file
    with open(validation_file_name, "rb") as file:
        validation_response = client.files.create(
            file=file, purpose="fine-tune"
        )
    validation_file_id = validation_response.id
    print("Validation file ID:", validation_file_id)

except Exception as e:
    print(f"Error uploading validation file: {e}")


Training file ID: file-3da8bb89e4944a37a951caf7d5ac82ff
Validation file ID: file-7e58ea9ecb6b4016a6962e1a45960997


In [8]:
# List all files
#check files status (needs to be Processed)
response = client.files.list()

# Iterate through the response to check the status of each file
for file in response:
    if file.id == training_file_id:
        print(f"Training file status: {file.status}")
    elif file.id == validation_file_id:
        print(f"Validation file status: {file.status}")



Validation file status: processed
Training file status: processed


Begin FineTuning

In [10]:
# Submit fine-tuning training job

response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-3.5-turbo"  # Change to a fine-tuneable model
)


job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

BadRequestError: Error code: 400 - {'error': {'code': 'invalidPayload', 'message': 'The specified model does not exist.'}}

List fine-tuning events

In [ ]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=5))

Final training run results

In [ ]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

Deployment

In [ ]:
# Deploy fine-tuned model

import json
import requests
load_dotenv()

token = os.getenv("token")
print(token)
subscription = os.getenv("subscription")
print(subscription)
resource_group = os.getenv("resource_group")
resource_name = os.getenv("resource_name")
model_deployment_name = os.getenv("model_deployment_name")

deploy_params = {'api-version': "2024-05-01-preview"}
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-35-turbo-0613.ft-3bf62ca6cb934fa299f25b03b9ead914", #retrieve this value from the previous call, it will look like gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

Test the Deployed Model

In [86]:
# Use the deployed customized model

import os
from openai import AzureOpenAI
load_dotenv()
import os

client = AzureOpenAI(
  azure_endpoint = os.getenv("OPENAI_API_BASE_fine_tuning"),
  api_key = os.getenv("OPENAI_API_KEY_fine_tuning"),
  api_version = "2024-05-01-preview",

)


response = client.chat.completions.create(
    model = "deployment-finetuning", # model = "Custom deployment name you chose for your fine-tuning model"
    messages = [
        {"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: couscous\n\nIngredients: [\"1 box powdered sugar\", \"8 oz. soft butter\", \"1 (8 oz.) peanut butter\", \"paraffin\", \"12 oz. chocolate chips\"]\n\nGeneric ingredients: "}, {"role": "assistant", "content": "[\"powdered sugar\", \"butter\", \"peanut butter\", \"paraffin\", \"chocolate chips\"]"}
    ]

)

print(response.choices[0].message.content)

["powdered sugar", "butter", "peanut butter", "paraffin", "chocolate chips"]
